In [3]:
# ==============================================================================
# TE Ponderar — Modo Laboratório (Malha Infinita)
# v0.3 (A_strict + A_soft + macro-métricas de saturação)
# ==============================================================================

from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional, Set
import math
import random

# ------------------------
# util matemático
# ------------------------
def log2(x: float) -> float:
    """Calcula logaritmo na base 2."""
    if x <= 0:
        return 0.0 # Define log(0) como 0 no contexto de entropia/informacao
    return math.log(x, 2)

def entropy_bits(counts: List[int]) -> float:
    """Calcula a Entropia (H) em bits (log2)."""
    n = sum(counts)
    if n <= 0:
        return 0.0
    H = 0.0
    for c in counts:
        if c <= 0:
            continue
        p = c / n
        H -= p * log2(p)
    return H

def log2_nCk(n: int, k: int) -> float:
    """Calcula log2(C(n,k)) - coeficiente binomial, via lgamma para estabilidade."""
    if k < 0 or k > n:
        return float("-inf")
    if k == 0 or k == n:
        return 0.0
    # Formula: log2(nCk) = (lgamma(n+1) - lgamma(k+1) - lgamma(n-k+1)) / log(2)
    return (math.lgamma(n+1) - math.lgamma(k+1) - math.lgamma(n-k+1)) / math.log(2)

# ------------------------
# regras determinísticas de split
# ------------------------
def apply_rule(rule: str, elems: List[int]) -> List[int]:
    """Aplica uma regra de divisao deterministica ao maior bloco."""
    m = len(elems)
    if m <= 1:
        return []

    # Regras de divisao (definição do SPLIT_RULE)
    if rule == "FIRST_HALF":
        return elems[: m//2]
    if rule == "SECOND_HALF":
        return elems[m//2 : ]
    if rule == "EVERY_OTHER":
        return elems[::2]
    if rule == "ODD_INDEX":
        return elems[1::2]
    if rule == "FIRST_QUARTER":
        return elems[: max(1, m//4)]
    if rule == "LAST_QUARTER":
        return elems[m - max(1, m//4):]
    if rule == "FIRST_THIRD":
        return elems[: max(1, m//3)]
    if rule == "MIDDLE_THIRD":
        a = max(1, m//3)
        b = max(1, m//3)
        start = a
        end = min(m, start + b)
        return elems[start:end]
    if rule == "LAST_THIRD":
        return elems[m - max(1, m//3):]
    if rule == "MOD3_0":
        return [x for i, x in enumerate(elems) if (i % 3) == 0]
    if rule == "MOD3_1":
        return [x for i, x in enumerate(elems) if (i % 3) == 1]
    if rule == "MOD3_2":
        return [x for i, x in enumerate(elems) if (i % 3) == 2]
    if rule == "FIRST_8":
        return elems[: min(8, m)]
    if rule == "LAST_8":
        return elems[max(0, m-8):]
    if rule == "EVERY_4":
        return [x for i, x in enumerate(elems) if (i % 4) == 0]
    if rule == "EVERY_5":
        return [x for i, x in enumerate(elems) if (i % 5) == 0]
    if rule == "ALT_CENTER":
        center = m//2
        out = []
        for d in range(m):
            j = center + (d//2 if d % 2 == 0 else -(d//2 + 1))
            if 0 <= j < m:
                if len(out) < (m//2):
                    out.append(elems[j])
        if not out:
            out = elems[:m//2]
        return out

    # fallback
    return elems[: m//2]

RULES = [
    "FIRST_HALF","SECOND_HALF","EVERY_OTHER","ODD_INDEX",
    "FIRST_QUARTER","LAST_QUARTER",
    "FIRST_THIRD","MIDDLE_THIRD","LAST_THIRD",
    "MOD3_0","MOD3_1","MOD3_2",
    "FIRST_8","LAST_8","EVERY_4","EVERY_5","ALT_CENTER"
]

# ------------------------
# estruturas de dados
# ------------------------
@dataclass
class Params:
    """Parâmetros do regime de simulacao (custos, N, passos)."""
    N: int = 64
    steps: int = 80
    seed: int = 7

    # rho (peso na complexidade R, a "pressao" para diversidade)
    rho: float = 0.35

    # CUSTOMIZACAO: SPLIT_RAND (necessário para evitar erro, mesmo que nao usado)
    rand_k_list: Tuple[int, ...] = (4, 8, 16)
    rand_samples_per_k: int = 1

    # custo "descrição/complexidade" (C)
    cP: float = 0.4 # Custo fixo para qualquer SPLIT (cP)
    cD: float = 0.05 # Custo logaritmico da complexidade de representar o SPLIT
    rule_space_cost: int = 24 # Espaco de regras (para SPLIT_RULE)

    # lambdas (peso da relacao na metrica I)
    lam_rel: float = 0.1
    lam_rel_size: float = 0.1 # bonus leve por aresta

    # custo REL (rel_base + rel_scale * log2(1 + E_now))
    rel_base: float = 0.005
    rel_scale: float = 0.005

    # barreira de aceitacao da Utilidade (0.0 = estrito; -0.05 = suave)
    acceptU_min: float = 0.0

    lab: bool = True
    tag: str = "A"
    log_path: Optional[str] = None

@dataclass
class State:
    """Estado do universo em um dado tempo (t)."""
    N: int
    markers: List[Set[int]]      # Q_n: cada marker é um conjunto de elementos (distinção)
    rel_edges: Set[Tuple[int, int]] # Relações (edges) entre indices de markers
    rel_deg: List[int]           # Graus por marker

@dataclass
class Candidate:
    """Evento candidato (SPLIT_RULE, REL, etc.) e seu custo."""
    kind: str
    desc: str
    cost: float
    payload: dict

# ------------------------
# partição e métricas básicas
# ------------------------
def signature_for_elem(e: int, markers: List[Set[int]]) -> Tuple[int, ...]:
    """Assinatura (binaria) de um elemento e em relacao a todos os markers."""
    return tuple(1 if e in mk else 0 for mk in markers)

def compute_blocks(state: State) -> Dict[Tuple[int, ...], List[int]]:
    """Calcula a particao (Blocos R) do universo N baseada nos markers."""
    blocks: Dict[Tuple[int, ...], List[int]] = {}
    for e in range(state.N):
        sig = signature_for_elem(e, state.markers)
        blocks.setdefault(sig, []).append(e)
    return blocks

def compute_H_R(state: State) -> Tuple[float, int]:
    """Calcula Entropia (H) e Numero de Blocos (R)."""
    blocks = compute_blocks(state)
    sizes = [len(v) for v in blocks.values() if len(v) > 0]
    R = len(sizes) # Numero de Espacos de Particao (Q_R)
    H = entropy_bits(sizes)
    return H, R

# ------------------------
# conectividade do grafo REL (macro-métricas)
# ------------------------
def component_sizes(num_nodes: int, edges: Set[Tuple[int,int]]) -> List[int]:
    """Calcula o tamanho dos componentes conectados (para o GCC)."""
    if num_nodes <= 0:
        return []
    adj = [[] for _ in range(num_nodes)]
    for (a,b) in edges:
        adj[a].append(b)
        adj[b].append(a)
    seen = [False]*num_nodes
    sizes = []
    for i in range(num_nodes):
        if seen[i]:
            continue
        # DFS
        stack = [i]
        seen[i] = True
        cnt = 0
        while stack:
            u = stack.pop()
            cnt += 1
            for v in adj[u]:
                if not seen[v]:
                    seen[v] = True
                    stack.append(v)
        sizes.append(cnt)
    sizes.sort(reverse=True)
    return sizes

def macro_stats(state: State, H: float, R: int, I: float) -> Dict[str, float]:
    """Métricas macro de saturação da malha (densidade, coesao, grau)."""
    m = len(state.markers)
    E = len(state.rel_edges)

    # Densidade: E / E_max
    dens = (2*E / (m*(m-1))) if m >= 2 else 0.0

    # Grau medio
    avg_deg = (2*E/m) if m > 0 else 0.0

    # GCC (Giant Component Connected)
    comps = component_sizes(m, state.rel_edges) if m > 0 else []
    gcc = comps[0] if comps else 0
    gcc_frac = (gcc/m) if m > 0 else 0.0

    # H norm
    Hnorm = (H / log2(R)) if R > 1 else 0.0

    return {
        "m": float(m),
        "E": float(E),
        "density": float(dens),
        "avg_deg": float(avg_deg),
        "gcc": float(gcc),
        "gcc_frac": float(gcc_frac),
        "components": float(len(comps)),
        "H": float(H),
        "R": float(R),
        "Hnorm": float(Hnorm),
        "I": float(I),
    }

# ------------------------
# métrica I (ontologia operacional)
# ------------------------
def compute_I(state: State, params: Params) -> float:
    """
    Calcula a Informacao Total (I) do sistema (Metrica I da TE).
    I = H + rho*log2(R) + lam_rel*log2(1 + E) + lam_rel_size * S
    """
    H, R = compute_H_R(state)
    E = len(state.rel_edges)

    # I: H + Ponderacao da diversidade R + Ponderacao das relacoes E
    I = H + params.rho * log2(max(1, R)) + params.lam_rel * log2(1 + E)

    # S: Soma bonus leve por aresta (size-bonus por aresta existente)
    if E > 0 and params.lam_rel_size != 0.0:
        s = 0.0
        for (a,b) in state.rel_edges:
            s += (min(len(state.markers[a]), len(state.markers[b])) / (2.0 * params.N))
        I += params.lam_rel_size * s

    return I

# ------------------------
# aplicar eventos (ações irreversíveis)
# ------------------------
def add_marker(state: State, new_set: Set[int]) -> None:
    """Aplica o SPLIT_RULE/RAND/SINGLETON (cria um novo Q)."""
    state.markers.append(set(new_set))
    state.rel_deg.append(0)

def add_rel(state: State, a: int, b: int) -> None:
    """Aplica o REL (cria uma nova relacao)."""
    if a == b:
        return
    if a > b:
        a, b = b, a
    if (a,b) in state.rel_edges:
        return
    state.rel_edges.add((a,b))
    state.rel_deg[a] += 1
    state.rel_deg[b] += 1

def apply_candidate(state: State, cand: Candidate) -> None:
    """Aplica o melhor candidato ao estado do universo."""
    if cand.kind in ("split_rule","split_rand","singleton"):
        add_marker(state, cand.payload["new_marker"])
    elif cand.kind == "rel":
        add_rel(state, cand.payload["a"], cand.payload["b"])

# ------------------------
# gerar candidatos
# ------------------------
def generate_candidates(state: State, params: Params, rng: random.Random) -> List[Candidate]:
    """Gera todos os eventos candidatos (SPLIT e REL) para o proximo passo."""
    blocks = compute_blocks(state)

    # Escolhe o maior bloco para ser dividido (o maior potencial para dI)
    sig_big, elems_big = max(blocks.items(), key=lambda kv: len(kv[1]))
    block_size = len(elems_big)

    cands: List[Candidate] = []

    # 1. SPLIT_RULE (Criação de Lei/Estrutura - Custo Alto e Deterministico)
    rule_cost = params.cP + params.cD * log2(params.rule_space_cost)
    for rule in RULES:
        subset = apply_rule(rule, elems_big)
        subset = list(dict.fromkeys(subset))
        if 1 <= len(subset) <= block_size - 1:
            cands.append(Candidate(
                kind="split_rule",
                desc=f"SPLIT_RULE(rule={rule},k=None, |S|={len(subset)}, block={block_size})",
                cost=rule_cost,
                payload={"new_marker": set(subset), "rule": rule, "block": block_size, "Ssize": len(subset)}
            ))

    # 2. SINGLETON (Divisao extrema - Custo Alto e Deterministico)
    if block_size >= 2:
        # Custo logaritmico do tamanho do bloco (escolher 1 de M)
        single_cost = params.cP + params.cD * log2(block_size)
        chosen = elems_big[0]
        cands.append(Candidate(
            kind="singleton",
            desc=f"SINGLETON(|S|=1, block={block_size})",
            cost=single_cost,
            payload={"new_marker": {chosen}, "block": block_size}
        ))

    # 3. SPLIT_RAND (Divisao aleatoria - Custo Variavel e Combinatorio)
    if block_size >= 4:
        for k in params.rand_k_list:
            if 2 <= k <= block_size - 2:
                for _ in range(params.rand_samples_per_k):
                    subset = rng.sample(elems_big, k)
                    # Custo log2(C(N, k))
                    rand_cost = params.cP + params.cD * log2_nCk(block_size, k)
                    cands.append(Candidate(
                        kind="split_rand",
                        desc=f"SPLIT_RAND(|S|={k}, block={block_size})",
                        cost=rand_cost,
                        payload={"new_marker": set(subset), "block": block_size, "Ssize": k}
                    ))

    # 4. REL (Criação de Relação/Conexão - Custo Baixo e C. Combinatorio)
    m = len(state.markers)
    if m >= 2:
        E_now = len(state.rel_edges)
        # Custo do REL cresce logaritmicamente com o número de relacoes existentes (E_now)
        rel_cost_base = params.rel_base + params.rel_scale * log2(1 + E_now)

        # Gera pares de markers ainda nao conectados (C(m, 2) possibilidades)
        for a in range(m):
            for b in range(a+1, m):
                if (a,b) in state.rel_edges:
                    continue
                cands.append(Candidate(
                    kind="rel",
                    desc=f"REL(a={a}, b={b})",
                    cost=rel_cost_base,
                    payload={"a": a, "b": b}
                ))

    return cands

# ------------------------
# simulação (LAB)
# ------------------------
def run_regime(params: Params) -> Tuple[State, List[str]]:
    """Funcao principal que executa um regime de simulacao."""
    rng = random.Random(params.seed)

    # Estado Inicial (Q0): um marcador singleton no universo N.
    state = State(
        N=params.N,
        markers=[{0}],
        rel_edges=set(),
        rel_deg=[0]
    )

    lines: List[str] = []

    def log_line(s: str):
        lines.append(s)
        print(s)

    header = (
        f"=== Regime {params.tag} [LAB] | N={params.N} steps={params.steps} seed={params.seed} | "
        f"rho={params.rho} acceptU_min>={params.acceptU_min} | "
        f"cP={params.cP} cD={params.cD} lam_rel={params.lam_rel} lam_rel_size={params.lam_rel_size} | "
        f"RELcost={params.rel_base}+{params.rel_scale}log2 ==="
    )
    log_line(header)

    H, R = compute_H_R(state)
    I = compute_I(state, params)
    ms = macro_stats(state, H, R, I)
    log_line(f"t=0  H={H:.4f}  R={R:3d}  I={I:.4f}  markers={len(state.markers)}  rel={len(state.rel_edges)}")
    log_line(f"      macro: dens={ms['density']:.4f}  <k>={ms['avg_deg']:.3f}  gcc={int(ms['gcc'])}/{int(ms['m'])}  Hnorm={ms['Hnorm']:.4f}")

    for t in range(1, params.steps+1):
        I0 = compute_I(state, params)
        H0, R0 = compute_H_R(state)

        cands = generate_candidates(state, params, rng)
        scored = []

        for cand in cands:
            # Cria cópia do estado para testar o evento
            st2 = State(
                N=state.N,
                markers=[set(x) for x in state.markers],
                rel_edges=set(state.rel_edges),
                rel_deg=list(state.rel_deg),
            )

            # Recalcula o custo do REL, pois ele depende do estado I0 (E_now)
            if cand.kind == "rel":
                E_now = len(state.rel_edges)
                cand = Candidate(
                    kind="rel",
                    desc=cand.desc,
                    cost=params.rel_base + params.rel_scale * log2(1 + E_now),
                    payload=cand.payload
                )

            apply_candidate(st2, cand)

            I1 = compute_I(st2, params)
            dI = I1 - I0
            U = dI - cand.cost # Utilidade = Ganho de Informacao - Custo
            scored.append((U, dI, cand.cost, cand, I1))

        # Escolhe o candidato de maior utilidade (U)
        scored.sort(key=lambda x: x[0], reverse=True)
        bestU, bestdI, bestC, bestCand, bestI1 = scored[0]

        # Regra de Parada (Barreira de Aceitacao)
        if bestU < params.acceptU_min:
            log_line(f"t={t} STOP(no_acceptable_utility) bestU={bestU}  H={H0:.4f} R={R0:3d} markers={len(state.markers)} rel={len(state.rel_edges)}")
            if params.lab:
                log_line("--- TOP 15 candidatos (U, dI, C) ---")
                for i, (U, dI, C, cand, _) in enumerate(scored[:15], 1):
                    log_line(f"{i:2d}. U={U:+.6f}  dI={dI:+.6f}  C={C:.6f}  [{cand.kind}]  {cand.desc}")
            break

        # Aplica o melhor
        apply_candidate(state, bestCand)

        H, R = compute_H_R(state)
        I = compute_I(state, params)
        ms = macro_stats(state, H, R, I)

        log_line(
            f"t={t}  ev={bestCand.desc:<55} dI={bestdI:+.4f}  C={bestC:.4f}  U={bestU:+.4f}  "
            f"H={H:.4f}  R={R:3d}  markers={len(state.markers)}  rel={len(state.rel_edges)}"
        )
        log_line(f"      macro: dens={ms['density']:.4f}  <k>={ms['avg_deg']:.3f}  gcc={int(ms['gcc'])}/{int(ms['m'])}  Hnorm={ms['Hnorm']:.4f}")

    # Salva log (se houver caminho)
    if params.log_path:
        with open(params.log_path, "w", encoding="utf-8") as f:
            f.write("\n".join(lines) + "\n")

    return state, lines

# ------------------------
# RODADAS DO EXPERIMENTO
# ------------------------

## 1) Regime A estrito (U >= 0.0)
A_strict = Params(
    N=64, steps=80, seed=7,
    rho=0.35,
    cP=0.4, cD=0.05,
    lam_rel=0.1, lam_rel_size=0.1,
    rel_base=0.005, rel_scale=0.005,
    acceptU_min=0.0,
    tag="A_strict",
    log_path="te_ponderar_lab_A_strict.txt",
    lab=True
)
print("--- Rodando Regime A_strict (Investimento Proibido) ---")
run_regime(A_strict)

print("\n" + "="*80 + "\n")

## 2) Regime A com barreira suave (U >= -0.05)
A_soft = Params(
    N=64, steps=200, seed=7,
    rho=0.35,
    cP=0.4, cD=0.05,
    lam_rel=0.1, lam_rel_size=0.1,
    rel_base=0.005, rel_scale=0.005,
    acceptU_min=-0.05,
    tag="A_soft",
    log_path="te_ponderar_lab_A_soft.txt",
    lab=True
)
print("--- Rodando Regime A_soft (Investimento Permitido) ---")
run_regime(A_soft)

print("\n" + "="*80 + "\n")

## 3) Regime B com custo REL alto (Teste de Robustez) - PRÓXIMO PASSO
B_soft = Params(
    N=64, steps=200, seed=7,
    rho=0.35,
    cP=0.4, cD=0.05,
    lam_rel=0.1, lam_rel_size=0.1,
    rel_base=0.05, # Aumenta 10x
    rel_scale=0.05, # Aumenta 10x
    acceptU_min=-0.05,
    tag="B_soft_HIGH_REL",
    log_path="te_ponderar_lab_B_soft.txt",
    lab=True
)
print("--- Rodando Regime B_soft_HIGH_REL (Custo de Relação 10x maior) ---")
run_regime(B_soft) # Descomente esta linha para rodar o proximo experimento!

print("\n[FIM DO CÓDIGO]")

--- Rodando Regime A_strict (Investimento Proibido) ---
=== Regime A_strict [LAB] | N=64 steps=80 seed=7 | rho=0.35 acceptU_min>=0.0 | cP=0.4 cD=0.05 lam_rel=0.1 lam_rel_size=0.1 | RELcost=0.005+0.005log2 ===
t=0  H=0.1161  R=  2  I=0.4661  markers=1  rel=0
      macro: dens=0.0000  <k>=0.000  gcc=1/1  Hnorm=0.1161
t=1  ev=SPLIT_RULE(rule=FIRST_HALF,k=None, |S|=31, block=63)    dI=+1.1889  C=0.6292  U=+0.5597  H=1.1003  R=  3  markers=2  rel=0
      macro: dens=0.0000  <k>=0.000  gcc=1/2  Hnorm=0.6942
t=2  ev=REL(a=0, b=1)                                           dI=+0.1008  C=0.0050  U=+0.0958  H=1.1003  R=  3  markers=2  rel=1
      macro: dens=1.0000  <k>=1.000  gcc=2/2  Hnorm=0.6942
t=3  ev=SPLIT_RULE(rule=FIRST_HALF,k=None, |S|=16, block=32)    dI=+0.6453  C=0.6292  U=+0.0160  H=1.6003  R=  4  markers=3  rel=1
      macro: dens=0.3333  <k>=0.667  gcc=2/3  Hnorm=0.8002
t=4  ev=REL(a=1, b=2)                                           dI=+0.0710  C=0.0100  U=+0.0610  H=1.6003  R=  4 